In [1]:
#poetry install python-dotenv
import pandas as pd
from sodapy import Socrata
import os
from dotenv import load_dotenv

load_dotenv('.env')
#TOKEN_APP = os.getenv("SECRET_TOKEN")
#TOKEN_USER = os.getenv("TOKEN_USER")
#TOKEN_PASS = os.getenv("TOKE_PASS")

client = Socrata(
        "www.datos.gov.co", None)
        #TOKEN_APP,
        #username= TOKEN_USER,
        #password= TOKEN_PASS,
        #timeout=5000)


consulta =  """SELECT tipo_de_contrato, nombre_entidad, fecha_de_firma
WHERE nombre_entidad = "FABRICA DE LICORES DEL TOLIMA"
limit 1000"""
data_id = "jbjy-vk9h"

results = client.get(data_id, query = consulta)
results_df = pd.DataFrame.from_records(results)

/var/folders/hs/q_ncl_2s737_06sj3c4ry41m0000gn/T/ipykernel_11977/4162852663.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
results_df.head()


,tipo_de_contrato,nombre_entidad,fecha_de_firma
0,Prestación de servicios,FABRICA DE LICORES DEL TOLIMA,2022-01-27T00:00:00.000
1,Suministros,FABRICA DE LICORES DEL TOLIMA,2023-02-14T00:00:00.000
2,Prestación de servicios,FABRICA DE LICORES DEL TOLIMA,2022-08-10T00:00:00.000
3,Suministros,FABRICA DE LICORES DEL TOLIMA,2023-08-16T00:00:00.000
4,Prestación de servicios,FABRICA DE LICORES DEL TOLIMA,2023-06-26T00:00:00.000
